In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import Video,display
from IPython.display import HTML
from base64 import b64encode
import matplotlib.animation as animation
from IPython.display import HTML
from PIL import Image
import seaborn as sns
%matplotlib inline

In [ ]:

root_path = r'/kaggle/input/uw-madison-gi-tract-image-segmentation'
train_path = os.path.join(root_path,'train.csv')
traindf = pd.read_csv(train_path)

print(traindf.shape)
traindf.head()

In [ ]:
import glob
def fx(x):
    p = x.split('_')
    caseid = p[0]
    day = p[1]
    idx = p[3]
    file_id = f'slice_{idx}'
    path = os.path.join(root_path,'train',f'{caseid}',f'{caseid}_{day}','scans')
    files = glob.glob(os.path.join(path,f'{file_id}*.png'))
    if len(files)==1:
        return files[0]
    else:
        print('more than one file',files)
        return files[0]

# Get all cases in Train & its images

In [ ]:
traindf['caseid'] = traindf['id'].map(lambda x : x.split('_')[0])
traindf['days'] = traindf['id'].map(lambda x : x.split('_')[1])
traindf['slice_id'] = traindf['id'].map(lambda x : x.split('_')[3])
traindf['imagepath'] = traindf['id'].map(fx)

In [ ]:
traindf.head(10)

In [ ]:
idx = np.random.choice(np.arange(traindf.shape[0]))
img = cv2.imread(traindf['imagepath'][idx],-1)
print(img.shape)
print(img.dtype)
img8 = np.uint8(255*((img-img.min())/(img.max()-img.min())))
plt.imshow(img8,cmap='gray')
plt.title(f'{idx}')
plt.show()

print(img.max())

# Video for a case with date

In [ ]:
def get_video_from_slices(df,caseid,date,do_normlize=True):
    # choose codec according to format needed
    _tdf = df[(df['caseid']==caseid) & (df['days']==date)]
    print(_tdf.shape)
    if _tdf.shape[0] > 0:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
        #get height width
        refimg = cv2.imread(_tdf['imagepath'][0],-1)
        H,W = refimg.shape
        video_path = './video.mp4'#/kaggle/working/video.mp4'
        video = cv2.VideoWriter(video_path, fourcc, 10, (W, H),0)
        for imgpath in _tdf['imagepath']:
            img = cv2.imread(imgpath,-1)
            if img.dtype=='uint16':
                if do_normlize:
                    img = np.uint8(255*((img-img.min())/(img.max()-img.min())))
                else:
                    img = np.uint8(255*(img/65535))
            if img.shape != refimg.shape:
                img = cv2.resize(img,(W,H),cv2.INTER_LINEAR)
#             __t = np.zeros([H,W,3],np.uint8)
#             __t[:,:,0]=img
#             __t[:,:,1]=img
#             __t[:,:,2]=img
            video.write(img)
        video.release()
        print('Done')
        return video_path
    else:
        print('No entry found')

def play_video(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=480 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return display(HTML(html))
  
    

In [ ]:
# inspired from taken from : https://www.kaggle.com/code/maunish/osic-super-cool-eda-and-pytorch-baseline
def get_animation_from_slices(df,caseid,date,do_normlize=True):
    # choose codec according to format needed
    _tdf = df[(df['caseid']==caseid) & (df['days']==date)]
    print(_tdf.shape)
    if _tdf.shape[0] > 0:
        fig = plt.figure()
        ims = list()
        for imgpath in _tdf['imagepath']:
            img = cv2.imread(imgpath,-1)
            if img.dtype=='uint16':
                if do_normlize:
                    img = np.uint8(255*((img-img.min())/(img.max()-img.min())))
                else:
                    img = np.uint8(255*(img/65535))
            image = plt.imshow(img,cmap='gray',animated=True)
            plt.axis('off')
            ims.append([image])
        ani = animation.ArtistAnimation(fig,ims,interval=100,blit=False,repeat_delay=1000)
        return ani
    else:
        print('No entry found')
        return None

In [ ]:
print(traindf['caseid'].unique())

In [ ]:
print(traindf['days'].unique())

In [ ]:
print(traindf[traindf['caseid']=='case138']['days'].unique())

In [ ]:
#_path = get_video_from_slices(traindf,'case138','day11')
#play_video(_path) 

In [ ]:
anim = get_animation_from_slices(traindf,'case138','day11')

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
traindf.head()

# Class distribution across slices

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=traindf,x='class');
traindf['class'].value_counts()

In [ ]:
plt.figure(dpi=100)
sns.countplot(data=traindf,x='caseid');
traindf['caseid'].unique().shape

# Class & days variation across cases 

In [ ]:
def plot_case_stats(df,caseid):
    dfc = df[df['caseid']==caseid]
    sns.countplot(x="days", hue="class", data=dfc)
    print(dfc.shape)
    
plot_case_stats(traindf,'case138')

In [ ]:
unqs= traindf['caseid'].unique()
np.random.choice(unqs)

In [ ]:
NCASE = 10
count = 0
unqs= traindf['caseid'].unique()
while count < NCASE:
    count +=1
    fig = plt.figure()
    caseid = np.random.choice(unqs)
    plot_case_stats(traindf,caseid)
    plt.title(f'{caseid}')
    

# Get annotation stats

In [ ]:
traindf['segmentation'].dropna().shape

# RLE encoding & decoding 

In [ ]:
#functions to convert encoding to mask and mask to encoding
def rle2mask(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


def mask2enc(mask, n=1):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1,n+1):
        p = (pixels == i).astype(np.int8)
        if p.sum() == 0: encs.append(np.nan)
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])[0] + 1
            runs[1::2] -= runs[::2]
            encs.append(' '.join(str(x) for x in runs))
    return encs

#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with transposed mask
def rle_encode_less_memory(img):
    #the image should be transposed
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

In [ ]:
segdf = traindf[traindf['segmentation'].notnull()]
cases = segdf['caseid'].unique()



**i.e every case has its marking **

# Mask visualization 

In [ ]:
import os,sys
do_normlize = True
for _cid in cases[1:2]:
    dft = segdf[segdf['caseid']==_cid]
    #print(dft.head())
    for _idx,row in dft.iterrows():
        #print(row)
        plt.figure(figsize=(15,11))
        _imgpath = row['imagepath']
        myimg = os.path.basename(_imgpath)
        img = cv2.imread(_imgpath,-1)
        if do_normlize:
            img = np.uint8(255*((img-img.min())/(img.max()-img.min())))
        else:
            img = np.uint8(255*(img/65535))
        parts = myimg.split('_')
        h,w = int(parts[2]),int(parts[3])
        pixh,pixw = float(parts[4]),float(parts[5].split('.')[0])
        _cls = row['class']
        #print(row['segmentation'])
        mask = rle2mask(row['segmentation'],(h,w))
        #print(mask.shape,mask.max(),mask.min())
        plt.subplot(1,3,1)
        plt.imshow(img,cmap='gray')
        plt.imshow(mask.T, alpha=0.7)
        plt.subplot(1,3,2)
        plt.imshow(img)
        plt.subplot(1,3,3)
        plt.imshow(mask.T)
        plt.title(_cls)
    plt.show()
        

# A lot Coming Soon